# Lab 4 - Create and load Gold layer

## 4.1 - Data model DDL

In [ ]:
-- Step 4.1.1
DROP TABLE IF EXISTS dbo.DimBrand;
DROP TABLE IF EXISTS dbo.DimStockItem;
DROP TABLE IF EXISTS dbo.DimCustomer;
DROP TABLE IF EXISTS dbo.FactSale;


In [ ]:
-- Step 4.1.2
CREATE TABLE [dbo].[DimBrand]
    (
        [BrandSK]                  [int]           NOT NULL,
        [BrandName]                [varchar](50)   NOT NULL,
        [TimeInserted]             [datetime2](6)  NOT NULL,
        [TimeLastChanged]          [datetime2](6)  NOT NULL,
        [IsDeleted]                [bit]           NOT NULL
    );
GO

CREATE TABLE [dbo].[DimStockItem]
(
    [StockItemSK]        [int]            NOT NULL,
    [StockItemSourceKey] [int]            NOT NULL,
    [StockItemName]      [varchar](100)   NULL,
    [BrandSK]            [int]            NOT NULL,
    [PackageTypeName]    [varchar](50)    NULL,
    [TaxRate]            [decimal](18, 3) NULL,
    [UnitPrice]          [decimal](18, 2) NULL,
    [Tags]               [varchar](max)   NULL,
    [TimeInserted]       [datetime2](6)   NOT NULL,
    [TimeLastChanged]    [datetime2](6)   NOT NULL,
    [IsDeleted]          [bit]            NOT NULL
);

CREATE TABLE [dbo].[DimCustomer]
(
    [CustomerSK]          [int]           NOT NULL,
    [CustomerSourceKey]   [int]           NOT NULL,
    [ContactFirstName]    [varchar](50)   NULL,
    [ContactLastName]     [varchar](50)   NULL,
    [PhoneNumber]         [varchar](20)   NULL,
    [WebsiteURL]          [varchar](256)  NULL,
    [DeliveryAddressLine] [varchar](60)   NULL,
    [DeliveryPostalCode]  [varchar](10)   NULL,
    [TimeInserted]        [datetime2](6)  NOT NULL,
    [TimeLastChanged]     [datetime2](6)  NOT NULL,
    [IsDeleted]          [bit]            NOT NULL
);
GO

CREATE TABLE [dbo].[FactSale]
(
    [SaleSK]         [int]                NOT NULL,
    [CustomerSK]     [int]                NOT NULL,
    [StockItemSK]    [int]                NOT NULL,
    
    [InvoiceID]      [int]                NOT NULL,
    [InvoiceDate]    [datetime2](6)       NULL,

    [Quantity]       [int]                NULL,
    [UnitPrice]      [decimal](18, 2)     NULL,
    [TaxRate]        [decimal](18, 3)     NULL, 
    [TaxAmount]      [decimal](18, 2)     NULL,
    [Profit]         [decimal](18, 2)     NULL,

    [TotalExcludingTax]  [decimal](18, 2) NULL, -- derrived
    [TotalIncludingTax]  [decimal](18, 2) NULL -- derrived
);
GO

CREATE OR ALTER VIEW dbo.vw_DimCustomer AS
SELECT *
FROM dbo.DimCustomer
WHERE IsDeleted = 0;
GO

CREATE OR ALTER VIEW dbo.vw_DimStockItem AS
SELECT *
FROM dbo.DimStockItem
WHERE IsDeleted = 0;
GO

CREATE OR ALTER VIEW dbo.vw_DimBrand AS
SELECT *
FROM dbo.DimBrand
WHERE IsDeleted = 0;
GO

CREATE OR ALTER VIEW dbo.vw_FactSales AS
SELECT      [SaleSK],
			[CustomerSK],
			[StockItemSK],
			[InvoiceID],
			[InvoiceDate],
			[Quantity],
			[UnitPrice],
			[TaxRate],
			[TaxAmount],
			[Profit],
			[TotalExcludingTax],
			[TotalIncludingTax]
FROM [WWI_Gold].[dbo].[FactSale]
GO

In [ ]:
-- Step 4.1.3: Confirm that objects are created
SELECT
    SCHEMA_NAME(schema_id) AS SchemaName,
    name AS TableName
FROM sys.tables
WHERE
    SCHEMA_NAME(schema_id) IN ('dbo')
    and name like 'Dim%'
ORDER BY
    SchemaName,
    TableName
GO

SELECT name, type, type_desc, create_date, modify_date
FROM sys.views
where is_ms_shipped=0;
GO

## 4.2 - Data transformation

### 4.2.1 - Create unknown member

In [ ]:
-- Step 4.2.1
DROP PROCEDURE IF EXISTS dbo.CreateUnknownMembers
GO
CREATE PROCEDURE dbo.CreateUnknownMembers
AS
BEGIN

    IF NOT EXISTS (SELECT * FROM dbo.DimBrand WHERE BrandSK = -1)
    INSERT INTO [dbo].[DimBrand]
           ([BrandSK],[BrandName], [TimeInserted], [TimeLastChanged],[IsDeleted]) 
    SELECT -1, 'Unknown', GETUTCDATE(), GETUTCDATE(), 0

    IF NOT EXISTS (SELECT * FROM dbo.DimCustomer WHERE CustomerSK = -1)
    INSERT INTO [dbo].[DimCustomer]
           ([CustomerSK]
           ,[CustomerSourceKey]
           ,[ContactFirstName]
           ,[ContactLastName]
           ,[PhoneNumber]
           ,[WebsiteURL]
           ,[DeliveryAddressLine]
           ,[DeliveryPostalCode]
           ,[TimeInserted] 
           ,[TimeLastChanged]
           ,[IsDeleted])
     SELECT -1, -1, 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', GETUTCDATE(), GETUTCDATE(), 0

    IF NOT EXISTS (SELECT * FROM dbo.DimStockItem WHERE StockItemSK = -1)
    INSERT INTO [dbo].[DimStockItem]
           ([StockItemSK]
           ,[StockItemSourceKey]
           ,[StockItemName]
           ,[BrandSK]
           ,[PackageTypeName]
           ,[TaxRate]
           ,[UnitPrice]
           ,[Tags]
           ,[TimeInserted] 
           ,[TimeLastChanged]
           ,[IsDeleted])
    SELECT -1, -1, 'Unknown', -1, 'Unknown', 0, 0, '["Unknown"]', GETUTCDATE(), GETUTCDATE(), 0

END
GO 
-- Check is stored procedure created
SELECT * FROM INFORMATION_SCHEMA.ROUTINES WHERE ROUTINE_TYPE = 'PROCEDURE' AND SPECIFIC_SCHEMA='dbo' and SPECIFIC_NAME='CreateUnknownMembers'
GO

### 4.2.2 - Create DimBrand load logic

In [ ]:
--Step 4.2.2.
DROP PROCEDURE IF EXISTS dbo.UpdateDimBrand
GO

CREATE PROCEDURE dbo.UpdateDimBrand
AS
BEGIN

    DECLARE @MaxID INT = (SELECT ISNULL(MAX(BrandSK), 0) FROM dbo.DimBrand);
    DECLARE @CurrentUTC DATETIME2= GETUTCDATE();

    WITH DistinctBrands AS (
        SELECT DISTINCT
            si.Brand AS BrandName
        FROM WWI_Silver.dbo.StockItems si
        WHERE si.Brand IS NOT NULL
    ),
    src AS (
        SELECT
            @MaxID + ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) AS BrandSK,
            BrandName
        FROM DistinctBrands
    )
    MERGE dbo.DimBrand T
    USING src AS S
        -- match on normalized business key
        ON T.BrandName = S.BrandName
    WHEN MATCHED AND (
        -- update only if something actually changed (e.g., casing/spacing)
        ISNULL(T.BrandName, '') <> ISNULL(S.BrandName, '')
    ) THEN
        UPDATE SET
            T.BrandName = S.BrandName,
            T.TimeLastChanged=@CurrentUTC
    WHEN NOT MATCHED BY TARGET THEN
        INSERT (BrandSK, BrandName, TimeInserted, TimeLastChanged, IsDeleted)
        VALUES (S.BrandSK, S.BrandName, @CurrentUTC, @CurrentUTC, 0);
END
GO

### 4.2.3 - Create DimStockItem load logic

In [ ]:
--Step 4.2.3
DROP PROCEDURE IF EXISTS dbo.UpdateDimStockItem
GO

CREATE PROCEDURE dbo.UpdateDimStockItem
AS
BEGIN

    DECLARE @MaxID int = (SELECT ISNULL(MAX(StockItemSK), 0) FROM dbo.DimStockItem);
    DECLARE @CurrentUTC DATETIME2= GETUTCDATE();

    ;WITH src AS
    (
        SELECT 
            StockItemSK = @MaxID + ROW_NUMBER() OVER (ORDER BY (SELECT NULL)),
            StockItemSourceKey = si.StockItemID,
            si.StockItemName,
            ISNULL(db.BrandSK, -1) AS BrandSK,
            si.PackageTypeName,
            si.TaxRate,
            si.UnitPrice,
            si.Tags
        FROM WWI_Silver.dbo.StockItems AS si
        LEFT JOIN dbo.DimBrand AS db
            ON db.BrandName = si.Brand
    )
    MERGE dbo.DimStockItem T
    USING (
        -- Align datatypes to target to ensure null-safe comparisons are accurate
        SELECT
            S.StockItemSK,
            S.StockItemSourceKey,
            S.StockItemName,
            BrandSK,
            S.PackageTypeName,
            S.TaxRate,
            S.UnitPrice,
            S.Tags
        FROM src AS S
    ) AS S
    ON T.StockItemSourceKey = S.StockItemSourceKey

    WHEN MATCHED AND (
        ISNULL(T.StockItemName,    '') <> ISNULL(S.StockItemName,    '')
        OR ISNULL(T.BrandSK,          '') <> ISNULL(S.BrandSK,          '')
        OR ISNULL(T.PackageTypeName,  '') <> ISNULL(S.PackageTypeName,  '')
        OR ISNULL(T.TaxRate,         0.0) <> ISNULL(S.TaxRate,         0.0)
        OR ISNULL(T.UnitPrice,       0.0) <> ISNULL(S.UnitPrice,       0.0)
        OR ISNULL(T.Tags,             '') <> ISNULL(S.Tags,             '')
    ) THEN
        UPDATE SET
            T.StockItemName     = S.StockItemName,
            T.BrandSK           = S.BrandSK,
            T.PackageTypeName   = S.PackageTypeName,
            T.TaxRate           = S.TaxRate,
            T.UnitPrice         = S.UnitPrice,
            T.Tags              = S.Tags,
            T.TimeLastChanged=@CurrentUTC

    WHEN NOT MATCHED BY TARGET THEN
        INSERT (
            StockItemSK,
            StockItemSourceKey,
            StockItemName,
            BrandSK,
            PackageTypeName,
            TaxRate,
            UnitPrice,
            Tags,
            TimeInserted,
            TimeLastChanged,
            IsDeleted
        )
        VALUES (
            S.StockItemSK,
            S.StockItemSourceKey,
            S.StockItemName,
            S.BrandSK,
            S.PackageTypeName,
            S.TaxRate,
            S.UnitPrice,
            S.Tags,
            @CurrentUTC,
            @CurrentUTC,
            0
        );
END;
GO



### 4.2.4 - Create DimCustomer load logic

In [ ]:
-- Step 4.2.4
DROP PROCEDURE IF EXISTS dbo.UpdateDimCustomer
GO

CREATE PROCEDURE dbo.UpdateDimCustomer
AS
BEGIN

    DECLARE @MaxID int = (SELECT ISNULL(MAX(CustomerSK), 0) FROM dbo.DimCustomer);
    DECLARE @CurrentUTC DATETIME2= GETUTCDATE();

    ;WITH src AS
    (
        SELECT
            CustomerSK = @MaxID + ROW_NUMBER() OVER (ORDER BY (SELECT NULL)),
            CustomerSourceKey = c.CustomerID,
            c.ContactFirstName,
            c.ContactLastName,
            c.PhoneNumber,
            c.WebsiteURL,
            c.DeliveryAddressLine,
            c.DeliveryPostalCode
        FROM WWI_Silver.dbo.Customers AS c
    )
    MERGE dbo.DimCustomer AS T
    USING src AS S
    ON T.CustomerSourceKey = S.CustomerSourceKey
    WHEN MATCHED AND (
        ISNULL(T.ContactFirstName,   '') <> ISNULL(S.ContactFirstName,   '')
        OR ISNULL(T.ContactLastName,    '') <> ISNULL(S.ContactLastName,    '')
        OR ISNULL(T.PhoneNumber,        '') <> ISNULL(S.PhoneNumber,        '')
        OR ISNULL(T.WebsiteURL,         '') <> ISNULL(S.WebsiteURL,         '')
        OR ISNULL(T.DeliveryAddressLine,'') <> ISNULL(S.DeliveryAddressLine,'')
        OR ISNULL(T.DeliveryPostalCode, '') <> ISNULL(S.DeliveryPostalCode, '')
    ) THEN
        UPDATE SET
            T.ContactFirstName     = S.ContactFirstName,
            T.ContactLastName      = S.ContactLastName,
            T.PhoneNumber          = S.PhoneNumber,
            T.WebsiteURL           = S.WebsiteURL,
            T.DeliveryAddressLine  = S.DeliveryAddressLine,
            T.DeliveryPostalCode   = S.DeliveryPostalCode,
            T.TimeLastChanged      = @CurrentUTC
    WHEN NOT MATCHED BY TARGET THEN
        INSERT (
            CustomerSK,
            CustomerSourceKey,
            ContactFirstName,
            ContactLastName,
            PhoneNumber,
            WebsiteURL,
            DeliveryAddressLine,
            DeliveryPostalCode,
            TimeInserted,
            TimeLastChanged,
            IsDeleted
        )
        VALUES (
            S.CustomerSK,
            S.CustomerSourceKey,
            S.ContactFirstName,
            S.ContactLastName,
            S.PhoneNumber,
            S.WebsiteURL,
            S.DeliveryAddressLine,
            S.DeliveryPostalCode,
            @CurrentUTC,
            @CurrentUTC,
            0
        );
END
GO

## 4.2.5 - Create FactSales load logic

In [ ]:
-- Step 4.2.5
DROP PROCEDURE IF EXISTS dbo.UpdateFactSale
GO

CREATE PROCEDURE dbo.UpdateFactSale
AS
BEGIN

    DECLARE @MaxID BIGINT = (SELECT ISNULL(MAX(SaleSK), 0) FROM dbo.FactSale)

    ;WITH src AS (
        SELECT
            @MaxID + ROW_NUMBER() OVER(ORDER BY (SELECT NULL)) AS [SaleSK],

            dc.[CustomerSK],
            dsi.[StockItemSK],

            fs.[InvoiceID],
            fs.[InvoiceDate],

            fs.[Quantity] AS [Quantity], -- Quantity sold
            fs.[UnitPrice], -- Unit price charged
            fs.[TaxRate], -- Tax rate applied
            fs.[TaxAmount], -- Tax amount
            fs.[LineProfit] AS [Profit], -- Profit made on the sale
            CAST(fs.[Quantity]*fs.[UnitPrice] AS decimal(18,2)) AS [TotalExcludingTax],
            CAST(fs.[Quantity]*fs.[UnitPrice] + fs.[TaxAmount] AS decimal(18,2)) AS [TotalIncludingTax]
        FROM WWI_Silver.dbo.FactSale fs
        LEFT JOIN dbo.vw_DimCustomer   dc  ON dc.CustomerSourceKey    = fs.CustomerID
        LEFT JOIN dbo.vw_DimStockItem  dsi ON dsi.StockItemSourceKey  = fs.StockItemID
    )
    MERGE dbo.FactSale T
    USING src AS S
        ON  T.InvoiceID   = S.InvoiceID
        AND T.StockItemSK = S.StockItemSK
        AND T.CustomerSK = S.CustomerSK
    WHEN MATCHED AND (
        ISNULL(T.InvoiceDate,  CAST('0001-01-01T00:00:00' AS datetime2(6)))
        <> ISNULL(S.InvoiceDate,  CAST('0001-01-01T00:00:00' AS datetime2(6)))
        OR 
        ISNULL(T.Quantity,   -1) <> ISNULL(S.Quantity,   -1)
        OR ISNULL(T.UnitPrice,    0.00) <> ISNULL(S.UnitPrice,    0.00)
        OR ISNULL(T.TaxRate,      0.000) <> ISNULL(S.TaxRate,      0.000)
        OR ISNULL(T.TaxAmount,    0.00) <> ISNULL(S.TaxAmount,    0.00)
        OR ISNULL(T.Profit,   0.00) <> ISNULL(S.Profit,   0.00)
        OR ISNULL(T.TotalExcludingTax,   0.00) <> ISNULL(S.TotalExcludingTax,   0.00)
        OR ISNULL(T.TotalIncludingTax,   0.00) <> ISNULL(S.TotalIncludingTax,   0.00)

    ) THEN
        UPDATE SET
            T.InvoiceDate = S.InvoiceDate
            , T.Quantity    = S.Quantity
            , T.UnitPrice   = S.UnitPrice
            , T.TaxRate     = S.TaxRate
            , T.TaxAmount   = S.TaxAmount
            , T.Profit  = S.Profit
            , T.TotalExcludingTax = S.TotalExcludingTax
            , T.TotalIncludingTax = S.TotalIncludingTax
    WHEN NOT MATCHED BY TARGET THEN
        INSERT (
            SaleSK
            , InvoiceDate
            , InvoiceID
            , CustomerSK
            , StockItemSK
            , Quantity
            , UnitPrice
            , TaxRate
            , TaxAmount
            , Profit
            , TotalExcludingTax
            , TotalIncludingTax
        )
        VALUES (
            S.SaleSK
            , S.InvoiceDate
            , S.InvoiceID
            , S.CustomerSK
            , S.StockItemSK
            , S.Quantity
            , S.UnitPrice
            , S.TaxRate
            , S.TaxAmount
            , S.Profit
            , S.TotalExcludingTax
            , S.TotalIncludingTax        
        );
END;
GO


## 4.2.6 - Validate that stored procedures are created

In [ ]:
-- Step 4.2.6
SELECT
    SCHEMA_NAME(p.schema_id) AS SchemaName,
    p.name AS ProcedureName, 
    p.create_date,
    p.modify_date
FROM
    sys.procedures AS p
WHERE
    SCHEMA_NAME(p.schema_id) = 'dbo'
ORDER BY
    SchemaName, ProcedureName

## 4.3 - Initial load

In [ ]:

-- Step 4.3
DECLARE @CountBeforeLoadDimCustomer  BIGINT = (SELECT COUNT_BIG(*) FROM dbo.DimCustomer)
DECLARE @CountBeforeLoadDimBrand     BIGINT = (SELECT COUNT_BIG(*) FROM dbo.DimBrand)
DECLARE @CountBeforeLoadDimStockItem BIGINT = (SELECT COUNT_BIG(*) FROM dbo.DimStockItem)
DECLARE @CountBeforeLoadFactSale     BIGINT = (SELECT COUNT_BIG(*) FROM dbo.FactSale)

EXEC dbo.CreateUnknownMembers;
EXEC dbo.UpdateDimBrand;
EXEC dbo.UpdateDimCustomer;
EXEC dbo.UpdateDimStockItem;
EXEC dbo.UpdateFactSale;

SELECT 'dbo'   AS SchemaName, 'DimBrand'       AS TableName, FORMAT(@CountBeforeLoadDimBrand,     'N0') AS RecordCount_BeforeLoad, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount_AfterLoad FROM dbo.DimBrand        UNION ALL
SELECT 'dbo'   AS SchemaName, 'DimCustomer'    AS TableName, FORMAT(@CountBeforeLoadDimCustomer,  'N0') AS RecordCount_BeforeLoad, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount_AfterLoad FROM dbo.DimCustomer     UNION ALL
SELECT 'dbo'   AS SchemaName, 'DimStockItem'   AS TableName, FORMAT(@CountBeforeLoadDimStockItem, 'N0') AS RecordCount_BeforeLoad, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount_AfterLoad FROM dbo.DimStockItem    UNION ALL
SELECT 'dbo'   AS SchemaName, 'FactSale'       AS TableName, FORMAT(@CountBeforeLoadFactSale,     'N0') AS RecordCount_BeforeLoad, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount_AfterLoad FROM dbo.FactSale
ORDER BY
    SchemaName,
    TableName
     

## 4.4 - Incremental load validation
To be used as part of Lab 4 and Lab 5

In [ ]:
SELECT 'gold'   AS MedallionLayer, 'DimBrand'        AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM dbo.DimBrand                                               UNION ALL
SELECT 'gold'   AS MedallionLayer, 'DimCustomer'     AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM dbo.DimCustomer                                            UNION ALL
SELECT 'gold'   AS MedallionLayer, 'DimStockItem'    AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM dbo.DimStockItem                                           UNION ALL
SELECT 'gold'   AS MedallionLayer, 'FactSale'        AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM dbo.FactSale                                               UNION ALL
SELECT 'gold'   AS MedallionLayer, 'FactSale - 2013' AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM dbo.FactSale WHERE YEAR(InvoiceDate) = 2013                UNION ALL
SELECT 'gold'   AS MedallionLayer, 'FactSale - 2014' AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM dbo.FactSale WHERE YEAR(InvoiceDate) = 2014                UNION ALL
SELECT 'gold'   AS MedallionLayer, 'FactSale - 2015' AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM dbo.FactSale WHERE YEAR(InvoiceDate) = 2015                UNION ALL
SELECT 'gold'   AS MedallionLayer, 'FactSale - 2016' AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM dbo.FactSale WHERE YEAR(InvoiceDate) = 2016                UNION ALL

SELECT 'silver' AS MedallionLayer, 'Customers  '     AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM WWI_Silver.dbo.Customers                                  UNION ALL
SELECT 'silver' AS MedallionLayer, 'DimStockItem'    AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM WWI_Silver.dbo.StockItems                                 UNION ALL
SELECT 'silver' AS MedallionLayer, 'FactSale'        AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM WWI_Silver.dbo.FactSale                                   UNION ALL
SELECT 'silver' AS MedallionLayer, 'FactSale - 2013' AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM WWI_Silver.dbo.FactSale WHERE YEAR(InvoiceDate) = 2013    UNION ALL
SELECT 'silver' AS MedallionLayer, 'FactSale - 2014' AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM WWI_Silver.dbo.FactSale WHERE YEAR(InvoiceDate) = 2014    UNION ALL
SELECT 'silver' AS MedallionLayer, 'FactSale - 2015' AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM WWI_Silver.dbo.FactSale WHERE YEAR(InvoiceDate) = 2015    UNION ALL
SELECT 'silver' AS MedallionLayer, 'FactSale - 2016' AS TableName, FORMAT(COUNT_BIG(*), 'N0') AS RecordCount FROM WWI_Silver.dbo.FactSale WHERE YEAR(InvoiceDate) = 2016
ORDER BY
    MedallionLayer,
    TableName